In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# 모델 로드
model_path = "./../project3/best_vggface_model_backup_epoch3.h5"  # 모델 경로
model = load_model(model_path, compile=False)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# 감정 클래스 리스트 (test_generator에서 가져온 class_indices 순서와 맞춰야 함)
emotion_classes = ["anger", "happy", "normal", "panic", "sadness"]

# OpenCV 얼굴 검출기 설정
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# 웹캠 시작
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일 변환 (얼굴 검출 정확도 향상을 위해)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # 얼굴 영역 추출
        face = frame[y:y + h, x:x + w]
        face_resized = cv2.resize(face, (224, 224))  # 모델 입력 크기로 조정
        face_array = np.expand_dims(face_resized / 255.0, axis=0)  # 정규화 및 차원 확장

        # 모델 예측
        predictions = model.predict(face_array)
        predicted_index = np.argmax(predictions)
        predicted_label = emotion_classes[predicted_index]

        # 결과 화면에 표시
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, predicted_label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # 화면에 출력
    cv2.imshow("Facial Expression Recognition", frame)

    # ESC 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == 27:
        break

# 웹캠 종료
cap.release()
cv2.destroyAllWindows()


OSError: Unable to synchronously open file (truncated file: eof = 152829952, sblock->base_addr = 0, stored_eof = 213129368)